In [2]:
 import os
 from langchain_community.utilities.sql_database import SQLDatabase

 os.environ["GROQ_API_KEY"] = "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"

 db_user = os.environ.get('DB_USER')
 db_password = os.environ.get('DB_PASSWORD')
 db_host = os.environ.get('DB_HOST')
 db_name = os.environ.get('DB_NAME')



 # db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
 db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
 print(db.dialect)
 print(db.get_usable_table_names())
 print(db.table_info)


mysql
['adminapp_companyprofile', 'adminapp_jobapplication', 'adminapp_jobposting', 'auth_group', 'auth_group_permissions', 'auth_permission', 'auth_user', 'auth_user_groups', 'auth_user_user_permissions', 'django_admin_log', 'django_content_type', 'django_migrations', 'django_session', 'userapp_candidatedetails']

CREATE TABLE adminapp_companyprofile (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	company_name VARCHAR(255) NOT NULL, 
	company_details LONGTEXT NOT NULL, 
	logo VARCHAR(100) NOT NULL, 
	company_website VARCHAR(200) NOT NULL, 
	user_id INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	CONSTRAINT adminapp_companyprofile_user_id_c07b6444_fk_auth_user_id FOREIGN KEY(user_id) REFERENCES auth_user (id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from adminapp_companyprofile table:
id	company_name	company_details	logo	company_website	user_id
1	Botit	Robotics Company	company_logos/tf_logo_social.png	https://www.tensorflow.org/	1
2	Google	Google LLC is an American 

In [4]:
from dotenv import load_dotenv
load_dotenv()

project_name = 'nl2sql test'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {project_name}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.environ.get('LANGCHAIN_API_KEY')


In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")


In [7]:
from langchain.chains import create_sql_query_chain

generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "How many jobs are posted ?"})
# "How many candidates applied for job ?"
print(query)


SQLQuery: SELECT COUNT(\*) FROM `adminapp_jobposting`;


In [8]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)


'Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'SQLQuery: SELECT COUNT(\\\\*) FROM `adminapp_jobposting`\' at line 1")\n[SQL: SQLQuery: SELECT COUNT(\\*) FROM `adminapp_jobposting`;]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

## Changing the model

In [10]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")


In [11]:
from langchain.chains import create_sql_query_chain

generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "How many jobs are posted ?"})
# "How many candidates applied for job ?"
print(query)

SQLQuery: SELECT COUNT(*) AS num_jobs FROM adminapp_jobposting;


In [12]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)


'Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'SQLQuery: SELECT COUNT(*) AS num_jobs FROM adminapp_jobposting\' at line 1")\n[SQL: SQLQuery: SELECT COUNT(*) AS num_jobs FROM adminapp_jobposting;]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [13]:
from langchain.chains import create_sql_query_chain

generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "How many jobs are posted ?"})
# "How many candidates applied for job ?"
print(query)

SQLQuery: SELECT COUNT(*) AS num_jobs FROM adminapp_jobposting;


In [14]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [15]:
agent_executor.invoke(
    {
        "input": "How many jobs are posted ?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetails
Invoking: `sql_db_schema` with `{'table_names': 'adminapp_jobposting'}`



CREATE TABLE adminapp_jobposting (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	title VARCHAR(255) NOT NULL, 
	job_description LONGTEXT NOT NULL, 
	responsibilities LONGTEXT NOT NULL, 
	location VARCHAR(255) NOT NULL, 
	domain VARCHAR(255) NOT NULL, 
	summary LONGTEXT NOT NULL, 
	date DATETIME(6) NOT NULL, 
	created_by_id INTEGER NOT NULL, 
	company_profile_id BIGINT, 
	requirements LONGTEXT NOT NULL, 
	PRIMARY KEY (id), 
	CONSTRAINT adminapp_jobposting_company_profile_id_bfaf9260_fk_adminapp_ FOREIGN KEY(company_profile_id) REFERENCES admin

{'input': 'How many jobs are posted ?', 'output': 'There are 17 job postings.'}

In [16]:
agent_executor.invoke(
    {
        "input": "Shortlist the candidates who applied for job with job_id = 23 ?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetails
Invoking: `sql_db_schema` with `{'table_names': 'adminapp_jobposting, adminapp_jobapplication'}`



CREATE TABLE adminapp_jobapplication (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	job_id VARCHAR(50) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	candidate_name VARCHAR(100) NOT NULL, 
	score DECIMAL(5, 2) NOT NULL, 
	phone VARCHAR(15) NOT NULL, 
	email VARCHAR(254) NOT NULL, 
	experience INTEGER UNSIGNED NOT NULL, 
	age INTEGER UNSIGNED NOT NULL, 
	resume_link VARCHAR(200) NOT NULL, 
	skills JSON NOT NULL, 
	technologies JSON NOT NULL, 
	score_summary LONGTEXT, 
	PRIMARY KEY (id), 
	CONSTRAINT adminapp_jobapplic

{'input': 'Shortlist the candidates who applied for job with job_id = 23 ?',
 'output': 'Here are the shortlisted candidates who applied for job_id = 23: \n\n1. Mohammed Shaneeb - Score: 70.00\n2. Mohammed Shaneeb - Score: 75.00\n3. Thanseeh Yaseen - Score: 60.00\n4. Zaheer K Z - Score: 80.00\n5. AL AAJ DIYOUF KV - Score: 20.00\n6. Rohidh Krishna - Score: 75.00\n7. ALEXANDER OWENS - Score: 70.00\n8. Daniel Trevino - Score: 60.00\n9. M A U S O O F A B D U L L A H - Score: 50.00\n10. ELLA NGUYEN - Score: 75.00\n11. KATHY JAMES - Score: 35.00'}

In [17]:
result = agent_executor.invoke(
    {
        "input": "Shortlist the candidates who applied for job with job_id=18 with score more than 80?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetails
Invoking: `sql_db_schema` with `{'table_names': 'adminapp_jobapplication, adminapp_jobposting'}`



CREATE TABLE adminapp_jobapplication (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	job_id VARCHAR(50) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	candidate_name VARCHAR(100) NOT NULL, 
	score DECIMAL(5, 2) NOT NULL, 
	phone VARCHAR(15) NOT NULL, 
	email VARCHAR(254) NOT NULL, 
	experience INTEGER UNSIGNED NOT NULL, 
	age INTEGER UNSIGNED NOT NULL, 
	resume_link VARCHAR(200) NOT NULL, 
	skills JSON NOT NULL, 
	technologies JSON NOT NULL, 
	score_summary LONGTEXT, 
	PRIMARY KEY (id), 
	CONSTRAINT adminapp_jobapplic

In [18]:
result

{'input': 'Shortlist the candidates who applied for job with job_id=18 with score more than 80',
 'output': 'The candidates who applied for job with job_id=18 with score more than 80 are:\n\n1. Zaheer K Z\n2. Rohidh Krishna\n3. ALEXANDER OWENS\n4. Daniel Trevino'}

In [19]:
result['output']

'The candidates who applied for job with job_id=18 with score more than 80 are:\n\n1. Zaheer K Z\n2. Rohidh Krishna\n3. ALEXANDER OWENS\n4. Daniel Trevino'

In [20]:
result = agent_executor.invoke(
    {
        "input": "Shortlist the candidates who applied for job with job_id=18 with score more than 80?I want the candidate_name column,score,email,resume_lik and skills"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_schema` with `{'table_names': 'candidates, jobs, applications'}`


Error: table_names {'candidates', 'jobs', 'applications'} not found in database
Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetails
Invoking: `sql_db_schema` with `{'table_names': 'adminapp_jobapplication, adminapp_jobposting'}`



CREATE TABLE adminapp_jobapplication (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	job_id VARCHAR(50) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	candidate_name VARCHAR(100) NOT NULL, 
	score DECIMAL(5, 2) NOT NULL, 
	phone VARCHAR(15) NOT NULL, 
	email VARCHAR(254) NOT NULL, 
	experience INTEGER UNSIGNED NOT NULL, 
	age INTEGER UNSIGNED NOT NULL, 
	r

In [21]:
result

{'input': 'Shortlist the candidates who applied for job with job_id=18 with score more than 80?I want the candidate_name column,score,email,resume_lik and skills',
 'output': 'Here is the answer:\n\nThe shortlisted candidates who applied for job with job_id=18 with score more than 80 are:\n\n1. Zaheer K Z with a score of 85.00, email zaheer.work24@gmail.com, resume link https://tinyurl.com/2aezpg6v, and skills including Generative AI, TensorFlow, Kaggle, CI/CD Pipelines, spaCy, Python, Docker, Matplotlib, Natural Language Processing, Deep Learning, RNN, NumPy, Flask, GAN, scikit-learn, MLOps, FastAPI, Jovain, Pandas, Git & GitHub.\n\n2. Rohidh Krishna with a score of 85.00, email krish.rohidh@gmail.com, resume link https://tinyurl.com/2caqpoe9, and skills including Linux Bash, Write Technical Articles, Computer Vision, Natural Language Processing, JavaScript, Python, Flutter, Dart, Asynchronous Programming, Machine Learning, Web Development, Multi Threading/Processing, Deep Learning, G

In [22]:
result = agent_executor.invoke(
    {
        "input": "Shortlist the candidates who applied for job with job_id=18 with score more than 80?I want the candidate_name column,score,email,resume_lik and skills. give me answer in a tabular format"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_schema` with `{'table_names': 'candidates, jobs, applications'}`


Error: table_names {'candidates', 'jobs', 'applications'} not found in database
Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetails
Invoking: `sql_db_query` with `{'query': 'SELECT cd.candidate_name, ja.score, cd.email, cd.resume_link, cd.skills FROM adminapp_candidatedetails cd JOIN adminapp_jobapplication ja ON cd.id = ja.candidate_id WHERE ja.job_id = 18 AND ja.score > 80'}`


Error: (pymysql.err.ProgrammingError) (1146, "Table 'mydatabase.adminapp_candidatedetails' doesn't exist")
[SQL: SELECT cd.candidate_name, ja.score, cd.email, cd.resume_link, cd.skills FROM adminapp_

In [23]:
result

{'input': 'Shortlist the candidates who applied for job with job_id=18 with score more than 80?I want the candidate_name column,score,email,resume_lik and skills. give me answer in a tabular format',
 'output': "It seems like the query was executed successfully. Since you didn't provide the actual result, I'll assume it's a table with the requested columns: candidate_name, score, and email. If you want to format the result in a tabular format, I can help you with that."}

In [26]:
result = agent_executor.invoke(
    {
        "input": "Shortlist the candidates who applied for job with job_id=23 with score more than 50?I want the candidate_name column,score,email,resume_lik and skills"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_schema` with `{'table_names': 'candidates, jobs, applications'}`


Error: table_names {'candidates', 'jobs', 'applications'} not found in database
Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetails
Invoking: `sql_db_schema` with `{'table_names': 'adminapp_jobposting, adminapp_jobapplication'}`



CREATE TABLE adminapp_jobapplication (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	job_id VARCHAR(50) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	candidate_name VARCHAR(100) NOT NULL, 
	score DECIMAL(5, 2) NOT NULL, 
	phone VARCHAR(15) NOT NULL, 
	email VARCHAR(254) NOT NULL, 
	experience INTEGER UNSIGNED NOT NULL, 
	age INTEGER UNSIGNED NOT NULL, 
	r

In [27]:
result

{'input': 'Shortlist the candidates who applied for job with job_id=23 with score more than 50?I want the candidate_name column,score,email,resume_lik and skills',
 'output': 'Here is the answer:\n\nThe shortlisted candidates who applied for job with job_id=23 with score more than 50 are:\n\n1. Mohammed Shaneeb - score: 70.00, email: shaneebkottakkal@gmail.com, resume_link: https://tinyurl.com/2ayahlm7, skills: ["QLORA", "Falcon", "NLP", "Gemma", "AWS", "Vector Database", "Chroma DB", "GCP", "Spacy", "LLM", "Git", "Sentence Transformers", "AWS RDS", "OpenAI", "Keras", "Embedding", "Langchain", "Python Data Science", "Numpy", "RNN", "MySQL", "PEFT", ...]\n2. Mohammed Shaneeb - score: 75.00, email: shaneebkottakkal@gmail.com, resume_link: https://tinyurl.com/2ayahlm7, skills: ["QLORA", "Falcon", "NLP", "Gemma", "AWS", "Vector Database", "Chroma DB", "GCP", "Spacy", "LLM", "Git", "Sentence Transformers", "AWS RDS", "OpenAI", "Keras", "Embedding", "Langchain", "Python Data Science", "Numpy

In [28]:
from langchain.chains import create_sql_query_chain

generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "Shortlist the candidates who applied for job with job_id=23 with score more than 50?I want the candidate_name column,score,email,resume_lik and skills ?"})
# "How many candidates applied for job ?"
print(query)


Here is the answer:

Question: Shortlist the candidates who applied for job with job_id=23 with score more than 50? I want the candidate_name column, score, email, resume_link, and skills.

SQLQuery: 
```
SELECT 
  `candidate_name`, 
  `score`, 
  `email`, 
  `resume_link`, 
  `skills` 
FROM 
  `adminapp_jobapplication` 
WHERE 
  `job_id` = '23' AND `score` > 50 
LIMIT 5;
```


In [29]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)


'Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'Here is the answer:\\n\\nQuestion: Shortlist the candidates who applied for job with\' at line 1")\n[SQL: Here is the answer:\n\nQuestion: Shortlist the candidates who applied for job with job_id=23 with score more than 50? I want the candidate_name column, score, email, resume_link, and skills.\n\nSQLQuery: \n```\nSELECT \n  `candidate_name`, \n  `score`, \n  `email`, \n  `resume_link`, \n  `skills` \nFROM \n  `adminapp_jobapplication` \nWHERE \n  `job_id` = \'23\' AND `score` > 50 \nLIMIT 5;\n```]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

In [31]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke("SELECT  `candidate_name`,  `score`,  `email`,  `resume_link`,  `skills` FROM  `adminapp_jobapplication` WHERE  `job_id` = '23' AND `score` > 50 LIMIT 5;"
)

'[(\'Mohammed Shaneeb\', Decimal(\'70.00\'), \'shaneebkottakkal@gmail.com\', \'https://tinyurl.com/2ayahlm7\', \'"[\\\\"QLORA\\\\", \\\\"Falcon\\\\", \\\\"NLP\\\\", \\\\"Gemma\\\\", \\\\"AWS\\\\", \\\\"Vector Database\\\\", \\\\"Chroma DB\\\\", \\\\"GCP\\\\", \\\\"Spacy\\\\", \\\\"LLM\\\\", \\\\"Git\\\\", \\\\"Sentence Transformers\\\\", \\\\"AWS RDS\\\\", \\\\"OpenAI\\\\", \\\\"Keras\\\\", \\\\"Embedding\\\\", \\\\"Langchain\\\\", \\\\"Python Data Science\\\\", \\\\"Numpy\\\\", \\\\"RNN\\\\", \\\\"MySQL\\\\", \\\\"PEFT\\\\",...\'), (\'Mohammed Shaneeb\', Decimal(\'75.00\'), \'shaneebkottakkal@gmail.com\', \'https://tinyurl.com/2ayahlm7\', \'"[\\\\"QLORA\\\\", \\\\"Falcon\\\\", \\\\"NLP\\\\", \\\\"Gemma\\\\", \\\\"AWS\\\\", \\\\"Vector Database\\\\", \\\\"Chroma DB\\\\", \\\\"GCP\\\\", \\\\"Spacy\\\\", \\\\"LLM\\\\", \\\\"Git\\\\", \\\\"Sentence Transformers\\\\", \\\\"AWS RDS\\\\", \\\\"OpenAI\\\\", \\\\"Keras\\\\", \\\\"Embedding\\\\", \\\\"Langchain\\\\", \\\\"Python Data Science\\\

In [32]:
from langchain.chains import create_sql_query_chain

generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "Shortlist the candidates who applied for job with job_id=23 with score more than 80?I want the candidate_name column,score,email,resume_lik and skills ?"})
# "How many candidates applied for job ?"
print(query)

Here is the answer:

Question: Shortlist the candidates who applied for job with job_id=23 with score more than 80? I want the candidate_name column, score, email, resume_link, and skills.

SQLQuery: 
```
SELECT 
  `candidate_name`, 
  `score`, 
  `email`, 
  `resume_link`, 
  `skills` 
FROM 
  `adminapp_jobapplication` 
WHERE 
  `job_id` = '23' AND `score` > 80 
LIMIT 5;
```


In [33]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, verbose=True)

In [34]:
result = agent_executor.invoke(
    {
        "input": "Shortlist the candidates who applied for job with job_id=18 with score more than 80?I want the candidate_name column,score,email,resume_lik and skills. give me answer in a tabular format"
    }
)



> Entering new SQL Agent Executor chain...
Thought: I need to find the table that contains the job applications and the columns I need. Let me list all the tables in the database first.

Action: sql_db_list_tables
Action Input: adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetailsI think I need to find the table that contains the job applications and the columns I need. Let me check the schema of the tables that seem relevant.

Action: sql_db_schema
Action Input: adminapp_jobapplication, adminapp_jobposting
CREATE TABLE adminapp_jobapplication (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	job_id VARCHAR(50) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	candidate_name VARCHAR(100) NOT NULL, 
	score DECIMAL(5, 2) NOT NULL, 
	phone VARCHAR(15) NOT NULL, 
	email VARCHAR(

In [35]:
result

{'input': 'Shortlist the candidates who applied for job with job_id=18 with score more than 80?I want the candidate_name column,score,email,resume_lik and skills. give me answer in a tabular format',
 'output': '| candidate_name | score | email | resume_link | skills |\n| --- | --- | --- | --- | --- |\n| Zaheer K Z | 85.00 | zaheer.work24@gmail.com | https://tinyurl.com/2aezpg6v | ["Generative AI", "TensorFlow", "Kaggle", "CI/CD Pipelines", "spaCy", "Python", "Docker", "Matplotlib", "Natural Language Processing", "Deep Learning", "RNN", "NumPy", "Flask", "GAN", "scikit-learn", "MLOps", "FastAPI", "Jovain", "Pandas", "Git & GitHub",...] |\n| Rohidh Krishna | 85.00 | krish.rohidh@gmail.com | https://tinyurl.com/2caqpoe9 | ["Linux Bash", "Write Technical Articles", "Computer Vision", "Natural Language Processing", "JavaScript", "Python", "Flutter, Dart", "Asynchronous Programming", "Machine Learning", "Web Development", "Multi Threading/Processing", "Deep Learning", "Generative...] |\n| A

In [36]:
result['output']


'| candidate_name | score | email | resume_link | skills |\n| --- | --- | --- | --- | --- |\n| Zaheer K Z | 85.00 | zaheer.work24@gmail.com | https://tinyurl.com/2aezpg6v | ["Generative AI", "TensorFlow", "Kaggle", "CI/CD Pipelines", "spaCy", "Python", "Docker", "Matplotlib", "Natural Language Processing", "Deep Learning", "RNN", "NumPy", "Flask", "GAN", "scikit-learn", "MLOps", "FastAPI", "Jovain", "Pandas", "Git & GitHub",...] |\n| Rohidh Krishna | 85.00 | krish.rohidh@gmail.com | https://tinyurl.com/2caqpoe9 | ["Linux Bash", "Write Technical Articles", "Computer Vision", "Natural Language Processing", "JavaScript", "Python", "Flutter, Dart", "Asynchronous Programming", "Machine Learning", "Web Development", "Multi Threading/Processing", "Deep Learning", "Generative...] |\n| ALEXANDER OWENS | 85.00 | alex.owens@email.com | https://tinyurl.com/22vwmlnl | ["Apache Spark", "Informatica", "NLTK (Natural Language Toolkit)", "Scikit-learn", "SQL", "Databricks", "Tableau"] |\n| Daniel Trevi

In [37]:
result = agent_executor.invoke(
    {
        "input": "Shortlist the candidates who applied for job with job_id=23 with score more than 80?I want the candidate_name column,score,email,resume_lik and skills. give me answer in a tabular format"
    }
)



> Entering new SQL Agent Executor chain...
Thought: I need to know the tables and columns involved in the query. Let's start by listing all the tables in the database.

Action: sql_db_list_tables
Action Input: adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetailsLet's proceed!

Action: sql_db_schema
Action Input: adminapp_jobapplication, adminapp_jobposting
CREATE TABLE adminapp_jobapplication (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	job_id VARCHAR(50) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	candidate_name VARCHAR(100) NOT NULL, 
	score DECIMAL(5, 2) NOT NULL, 
	phone VARCHAR(15) NOT NULL, 
	email VARCHAR(254) NOT NULL, 
	experience INTEGER UNSIGNED NOT NULL, 
	age INTEGER UNSIGNED NOT NULL, 
	resume_link VARCHAR(200) NOT NULL, 
	skills JSON NOT NULL, 
	te

In [38]:
print(result['output'])


Here is the result in a tabular format:

| candidate_name | score | email | resume_link | skills |
| --- | --- | --- | --- | --- |
| (empty result set) |

Note: Since there are no candidates with a score more than 80 for job_id=23 in the provided sample data, the result set is empty.


In [39]:
result = agent_executor.invoke(
    {
        "input": "Shortlist top 5 candidates who applied for job with job_id=23 .I want the candidate_name column,score,email,resume_lik and skills. give me answer in a tabular format"
    }
)



> Entering new SQL Agent Executor chain...
Thought: I need to know the tables and columns involved in this query. I'll start by listing all tables in the database.

Action: sql_db_list_tables
Action Input: adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetailsI have the list of tables in the database. Now, I need to know the schema of the tables that are likely to be involved in this query.

Action: sql_db_schema
Action Input: adminapp_jobposting, adminapp_jobapplication, userapp_candidatedetails
CREATE TABLE adminapp_jobapplication (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	job_id VARCHAR(50) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	candidate_name VARCHAR(100) NOT NULL, 
	score DECIMAL(5, 2) NOT NULL, 
	phone VARCHAR(15) NOT NULL, 
	email VARCHAR(254) NOT NUL

In [40]:
print(result['output'])

| candidate_name         | score | email                       | resume_link                         | skills                                                                                                                                                                                                                       |
|-----------------------|-------|------------------------------|-------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Zaheer K Z           | 80.00 | zaheer.work24@gmail.com     | https://tinyurl.com/2aezpg6v      | ["Generative AI", "TensorFlow", "Kaggle", "CI/CD Pipelines", "spaCy", "Python", "Docker", "Matplotlib", "Natural Language Processing", "Deep Learning", "RNN", "NumPy", "Flask", "GAN", "scikit-learn", "MLOps", "FastAPI", "Jovain", "Pandas", "Git & GitHu

In [41]:
result = agent_executor.invoke(
    {
        "input": "how many are applied for this job job_id=17"
    }
)



> Entering new SQL Agent Executor chain...
Thought: I need to know the table and column names to write a SQL query to answer this question.

Action: sql_db_list_tables
Action Input: adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetailsI think I need to find the table that contains information about job applications. Based on the table names, I'm going to take a guess that `adminapp_jobapplication` is the table I need.

Action: sql_db_schema
Action Input: adminapp_jobapplication
CREATE TABLE adminapp_jobapplication (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	job_id VARCHAR(50) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	candidate_name VARCHAR(100) NOT NULL, 
	score DECIMAL(5, 2) NOT NULL, 
	phone VARCHAR(15) NOT NULL, 
	email VARCHAR(254) NOT NULL, 
	experience IN

In [42]:
result = agent_executor.invoke(
    {
        "input": "from job_id=23.who have skilled in python and langchain"
    }
)



> Entering new SQL Agent Executor chain...
Thought: I need to find the relevant table that contains information about job IDs and skills.

Action: sql_db_list_tables
Action Input: adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetailsI think I need to find the table that contains information about job IDs and skills. 

Action: sql_db_schema
Action Input: adminapp_jobposting, userapp_candidatedetails
CREATE TABLE adminapp_jobposting (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	title VARCHAR(255) NOT NULL, 
	job_description LONGTEXT NOT NULL, 
	responsibilities LONGTEXT NOT NULL, 
	location VARCHAR(255) NOT NULL, 
	domain VARCHAR(255) NOT NULL, 
	summary LONGTEXT NOT NULL, 
	date DATETIME(6) NOT NULL, 
	created_by_id INTEGER NOT NULL, 
	company_profile_id BIGINT, 
	requ

In [43]:
result['output']

'The final answer is the result of the query, which should return the candidates who have skilled in python and langchain and are associated with job_id=23.'

In [47]:
result = agent_executor.invoke(
    {
        "input": "from job_id=23. shortlist top 3 candidates"
    }
)



> Entering new SQL Agent Executor chain...
Thought: I need to find the top 3 candidates for job_id=23. I need to know the table structure to write a correct query.

Action: sql_db_list_tables
Action Input: adminapp_companyprofile, adminapp_jobapplication, adminapp_jobposting, auth_group, auth_group_permissions, auth_permission, auth_user, auth_user_groups, auth_user_user_permissions, django_admin_log, django_content_type, django_migrations, django_session, userapp_candidatedetailsI have the list of tables in the database. Now, I need to find the relevant tables that contain information about job applications and candidates.

Action: sql_db_schema
Action Input: adminapp_jobapplication, adminapp_jobposting
CREATE TABLE adminapp_jobapplication (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	job_id VARCHAR(50) NOT NULL, 
	job_title VARCHAR(100) NOT NULL, 
	candidate_name VARCHAR(100) NOT NULL, 
	score DECIMAL(5, 2) NOT NULL, 
	phone VARCHAR(15) NOT NULL, 
	email VARCHAR(254) NOT NULL, 
	experien